# Weak References - Part 2 of 2 (the weird stuff)

## Review: Other proxy object in the standard library

We've covered two other kinds of proxy objects:

### Mapping proxies

The `__dict__` descriptor on a class returns a mapping proxy, which prohibits mutating operations such as setting a different value for a key:

In [1]:
list.__dict__

mappingproxy({'__new__': <function list.__new__(*args, **kwargs)>,
              '__repr__': <slot wrapper '__repr__' of 'list' objects>,
              '__hash__': None,
              '__getattribute__': <slot wrapper '__getattribute__' of 'list' objects>,
              '__lt__': <slot wrapper '__lt__' of 'list' objects>,
              '__le__': <slot wrapper '__le__' of 'list' objects>,
              '__eq__': <slot wrapper '__eq__' of 'list' objects>,
              '__ne__': <slot wrapper '__ne__' of 'list' objects>,
              '__gt__': <slot wrapper '__gt__' of 'list' objects>,
              '__ge__': <slot wrapper '__ge__' of 'list' objects>,
              '__iter__': <slot wrapper '__iter__' of 'list' objects>,
              '__init__': <slot wrapper '__init__' of 'list' objects>,
              '__len__': <slot wrapper '__len__' of 'list' objects>,
              '__getitem__': <method '__getitem__' of 'list' objects>,
              '__setitem__': <slot wrapper '__setitem__' of

We can also construct mapping proxies explicitly using the convenient `MappingProxyType` alias in the `types` module.

In [2]:
import types

In [3]:
dir(types)

['AsyncGeneratorType',
 'BuiltinFunctionType',
 'BuiltinMethodType',
 'CellType',
 'ClassMethodDescriptorType',
 'CodeType',
 'CoroutineType',
 'DynamicClassAttribute',
 'EllipsisType',
 'FrameType',
 'FunctionType',
 'GeneratorType',
 'GenericAlias',
 'GetSetDescriptorType',
 'LambdaType',
 'MappingProxyType',
 'MemberDescriptorType',
 'MethodDescriptorType',
 'MethodType',
 'MethodWrapperType',
 'ModuleType',
 'NoneType',
 'NotImplementedType',
 'SimpleNamespace',
 'TracebackType',
 'UnionType',
 'WrapperDescriptorType',
 '_GeneratorWrapper',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_calculate_meta',
 '_cell_factory',
 'coroutine',
 'new_class',
 'prepare_class',
 'resolve_bases']

In [4]:
help(types.MappingProxyType)

Help on class mappingproxy in module builtins:

class mappingproxy(object)
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __ior__(self, value, /)
 |      Return self|=value.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __len__(self, /)
 |      Return len(self).
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __or__(self, value, /)
 |      Return self|value.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __reversed__(...)
 |      D.__reversed__() -> r

In [5]:
d = {1: 1, 2: 2, 3: 3}

In [6]:
md = types.MappingProxyType(d)

In [7]:
md 

mappingproxy({1: 1, 2: 2, 3: 3})

In [8]:
md[2]

2

In [9]:
md[2] = 3

TypeError: 'mappingproxy' object does not support item assignment

In [10]:
d[2] = 4

In [11]:
md[2]  # It's a proxy, so it reflects changes to the object "behind" it.

4

### `super` proxies

In [12]:
class Base: 
    def f(self):
        print("Base.f")

In [13]:
class Derived(Base): 
    def f(self): 
        print("Derived.f")
        super().f()

In [14]:
Derived().f()

Derived.f
Base.f


## Weak reference proxies

In [15]:
import weakref

In [16]:
class C: 
    def __init__(self): 
        self.a = 'a'
        self.b = 'b'

In [17]:
c = C()

In [18]:
wcsimple = weakref.proxy(c)

In [19]:
c.a

'a'

In [20]:
wcsimple.a

'a'

In [21]:
del c

In [22]:
wcsimple.a

ReferenceError: weakly-referenced object no longer exists

In [23]:
c = C()

In [24]:
c.a

'a'

In [25]:
wc = weakref.proxy(c, lambda x: print(f"{x}'s referant has died."))

In [26]:
wc

In [27]:
wc.a

'a'

In [28]:
del c

Exception ignored in: <function <lambda> at 0x00000215E1017880>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_21124\3266874697.py", line 1, in <lambda>
ReferenceError: weakly-referenced object no longer exists


In [30]:
wc

ReferenceError: weakly-referenced object no longer exists

In [31]:
c = C()

In [32]:
cghost = weakref.proxy(c, lambda x: print("Boooooooooooooooo!"))

In [33]:
del c

Boooooooooooooooo!


In [34]:
c = C()

In [35]:
rptest = weakref.proxy(c)

In [36]:
repr(c)

'<__main__.C object at 0x00000215E11C7BE0>'

In [37]:
repr(rptest)

'<weakproxy at 0x00000215DFB173D0 to C at 0x00000215E11C7BE0>'

In [38]:
rptest

In [39]:
rptest.__repr__()

'<__main__.C object at 0x00000215E11C7BE0>'

In [40]:
class DiffBox: 
    
    def __init__(self, value): 
        self.value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'

    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return self.value == other.value
    
    def __sub__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return type(self)(self.value - other.value)

In [41]:
d1 = DiffBox(1)

In [42]:
d5 = DiffBox(5)

In [43]:
d4 = d5 - d1

In [46]:
d4

DiffBox(4)

In [44]:
d5 == DiffBox(5)

True

In [45]:
d5 == DiffBox(4)

False